In [1]:
import pandas as pd
import geopy as gp
from geopy import distance
import re
import datetime

In [2]:
# change path back to original file name; changed out of laziness
df = pd.read_excel("schnitzler_places.xlsx")


In [3]:
df.columns

Index(['date', 'cert', 'place', 'geonames', 'lat', 'long', 'pmb', 'akon',
       'note1', 'note2', 'note3', 'note4', 'note5'],
      dtype='object')

In [4]:
test = ["Theater" in el for el in df.date]

In [5]:
# drop the rows with theater as date
df = df[[ not el for el in test]]

In [6]:
df["date_date"] = pd.to_datetime(df.date)

In [7]:
def iterate_dataframe(dataframe, distance_per_day):
    """
    distance_per_day (int): give a number that represents the max distance (in kilometer "luftlinie") 
    between two geogpraphical points. Used to calculate a threshold to test the calculated distance per day against.
    dataframe: a pandas DataFrame.
    """
    df = dataframe
    res = []
    
    def get_time_sep(date1, date2):
        """
        helper function to calculate the distance between two timestamps in days.
        """
        return (date1 - date2).days

    def get_distance(a, b):
        """
        helper function to calculate the distance between two geographical points in km
        """
        return int(gp.distance.distance(a,b).km)

    for idx, el in df.iterrows():
        if len(df) >= idx >= 1:
            last = df.iloc[idx-1]
            
            # Try-Except needed because there are malformed floats (strings) in the lat-long columns
            try: 
                
                # days between a given row and the row before
                time_delta_in_days = get_time_sep(el.date_date, last.date_date)
                # distance in km between the lat-long coordinates in one row and the row before
                distance_in_km = get_distance((el.lat, el.long), (last.lat, last.long))
                
                # tolerated_time_distance: f.e.: if you allow 300km per day, a distance of 600km for a two day-trip is accepted.
                # abs is used here, because some timedeltas are negative (as we check against the entry before)
                tolerated_time_distance = abs(time_delta_in_days * distance_per_day)
                
                # some timedelta where 0 and produce a division by zero error, that is caught here
                if time_delta_in_days != 0:
                    # abs is used here, because some timedeltas are negative (as we check against the entry before)
                    calculated_time_distance = abs(int(distance_in_km / time_delta_in_days))
                    
                    # the juicy part: if the calculated distance is bigger then the threshold, do your thing:
                    if tolerated_time_distance < calculated_time_distance and el.place != last.place:
                        #print(el.date, el.place, last.date, last.place)
                        #print(idx, idx-1, "tolerated_time_distance", tolerated_time_distance, "km per tag: ", calculated_time_distance, "tage:", time_delta_in_days, "distance:", distance_in_km)
                        res.append({"idx_from":idx-1,"from":last.place, "idx_to":idx, "to":el.place, "km_per_day":calculated_time_distance, "threshold":distance_per_day, "date_from":last.date, "date_to":el.date})
            except Exception as e:
                # Print the malformed lat-long columns and all other occuring errors with the row on which the script fails.
                print(f"Error in row: {idx}. Message: {e}")
                continue
    return res

In [8]:
res = iterate_dataframe(df, 300)

/Users/oldfiche/.pyenv/versions/3.10.4/lib/python3.10/site-packages/geopy/point.py:472: UserWarning: Latitude normalization has been prohibited in the newer versions of geopy, because the normalized value happened to be on a different pole, which is probably not what was meant. If you pass coordinates as positional args, please make sure that the order is (latitude, longitude) or (y, x) in Cartesian terms.
  return cls(*args)


Error in row: 8120. Message: Latitude must be in the [-90; 90] range.
Error in row: 8121. Message: Latitude must be in the [-90; 90] range.
Error in row: 8122. Message: Latitude must be in the [-90; 90] range.
Error in row: 8123. Message: Latitude must be in the [-90; 90] range.
Error in row: 8125. Message: Latitude must be in the [-90; 90] range.
Error in row: 8126. Message: Latitude must be in the [-90; 90] range.
Error in row: 8127. Message: Latitude must be in the [-90; 90] range.
Error in row: 8128. Message: Latitude must be in the [-90; 90] range.
Error in row: 8129. Message: Latitude must be in the [-90; 90] range.
Error in row: 8130. Message: Latitude must be in the [-90; 90] range.
Error in row: 8545. Message: Latitude must be in the [-90; 90] range.
Error in row: 8546. Message: Latitude must be in the [-90; 90] range.
Error in row: 8548. Message: Latitude must be in the [-90; 90] range.
Error in row: 8549. Message: Latitude must be in the [-90; 90] range.
Error in row: 9114. 

In [9]:
tf = pd.DataFrame(res)

In [10]:
tf

,idx_from,from,idx_to,to,km_per_day,threshold,date_from,date_to
0,244,Frankfurt am Main,245,Wien,599,300,1879-08-30,1879-08-31
1,272,Mainz,273,Scheveningen,366,300,1879-09-13,1879-09-14
2,275,München,276,Mainz,316,300,1879-09-14,1879-09-15
3,1393,Wien,1394,Ragaz,533,300,1882-07-30,1882-07-31
4,2048,Wien,2049,Fiume,352,300,1884-04-09,1884-04-10
...,...,...,...,...,...,...,...,...
305,22172,Berlin,22173,Wien,524,300,1927-11-28,1927-11-27
306,22185,Wien,22186,Berlin,524,300,1927-12-11,1927-12-10
307,22320,Wien,22321,Istanbul,1276,300,1928-04-23,1928-04-22
308,22322,Wien,22323,Istanbul,1276,300,1928-04-24,1928-04-23


In [11]:
tf.to_csv("output_schnitzler_places.csv", index=False, encoding="utf-8")

In [12]:
# I used this to test for string-junk in the date column
for el in df.date:
    test = re.search(r"[A-Za-z]", el)
    if test:
        print(test.group(0), el)